### 4. Generation of the spectral index cubes


In [ ]:
import os
import glob

import numpy as np
import pandas as pd

import rasterio as rio
import rep_cbers_cube.vegetation_index as vi

from loguru import logger
logger.add("logs/cube_vegetation_index_{time}.log")

**Parameters**

In [ ]:
#
# directory where cropped raster is in
#
input_raster_path = ""

#
# directory where vegetation index will be write
#
output_cubes_vegetation_index = ""

#
# other definitions
#
factor = 10000
na_values = -9999


**General configurations**

In [ ]:
#
# define cropped rasters
#
cropped_cubes = glob.glob(os.path.join(input_raster_path, "*.tif"))

#
# create output directory
#
os.makedirs(output_cubes_vegetation_index, exist_ok=True)


**Temporal definitions**

In [ ]:
#
# extract start-end dates from cube timeline (Using filenames pattern)
#
start_date = list(map(lambda x: x.split('_')[-3], cropped_cubes))
end_date   = list(map(lambda x: x.split('_')[-2], cropped_cubes))

#
# defining output object
#
results = {
    'src_cube': [], 
    'dst_cube': [], 
    'start_date': start_date,
    'end_date': end_date
}

**Generating the index cubes**

The indexes `GEMI`, `GNDVI`, `NDWI2` and `PVR` are calculated

In [ ]:
for cropped_cube in cropped_cubes:

    """
    Band order:
         'EVI', 'NDVI', 'BAND13', 'BAND14', 'BAND15', 'BAND16'
    """    
    
    logger.info(f"Processing {cropped_cube}")
    
    #
    # general definitions (input/output pattern)
    #
    raster_out = os.path.split(cropped_cube)[-1]
    raster_out = f"{os.path.splitext(raster_out)[0]}_vegetation_index.tif"
    raster_out = os.path.join(output_cubes_vegetation_index, raster_out)
    
    results["src_cube"].append(cropped_cube)
    results["dst_cube"].append(raster_out)

    ds = rio.open(cropped_cube)
    cubedata = ds.read() / factor
    
    #
    # GEMI Index
    #
    gemi = vi.gemi(cubedata[4, :, :], cubedata[5, :, :])
    
    #
    # GNDVI
    #
    gndvi = vi.gndvi(cubedata[5, :, :], cubedata[3, :, :])
    
    #
    # NDWI2
    #
    ndwi2 = vi.ndwi2(cubedata[3, :, :], cubedata[5, :, :])
    
    #
    # PVR
    #
    pvr = vi.pvr(cubedata[3, :, :], cubedata[4, :, :])
    
    ds_profile = ds.profile.copy()
    ds_profile.update(dtype = 'int16', count = 4)
    
    #
    # write brick
    #
    with rio.open(raster_out, 'w', **ds_profile) as vegetation_raster:
        for band, idx in zip([gemi, gndvi, ndwi2, pvr], range(1, 5)):
            vegetation_raster.write((band * factor).astype('int16'), idx)


**Write bricks index**

In [ ]:
pd.DataFrame(results).to_csv(
        os.path.join(output_cubes_vegetation_index, "cropped_cube_vegetationindex.csv"), 
    index = False
)